In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split #separate into train test data
from sklearn.preprocessing import OneHotEncoder
import tensorflow
from keras import models
from keras import layers
print("Load completed")

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Load completed


In [2]:
#Read the datas from csv files
train = pd.read_csv('/kaggle/input/playground-series-s3e16/train.csv', index_col=0)
test = pd.read_csv('/kaggle/input/playground-series-s3e16/test.csv', index_col=0)

#Separate train data into X and y
X = train.iloc[:,:-1]
y = train.iloc[:,-1]

#Create a encoder
encoder = OneHotEncoder(sparse=False)

#Encode the sex with using OneHot Encoder
sex_data = encoder.fit_transform(X['Sex'].values.reshape(-1,1))
sex_df = pd.DataFrame(sex_data, columns=['Sex_F','Sex_I','Sex_M'])
sex_df = sex_df.set_index(X.index)

#Change the sex column on train data
X.drop('Sex',inplace=True,axis=1)
X = pd.concat([sex_df,X],axis=1)

#Separate X and y into train test splits
X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.2, random_state=42)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [3]:
#Create the model
model = models.Sequential()

#Add layers into model
model.add(layers.Dense(32, activation='relu', input_shape=[10]))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1))

#Compile model
model.compile(optimizer='rmsprop',
             loss='mse')

In [4]:
# Fit model
model.fit(X_train, y_train,
          batch_size=32, 
          epochs=100, 
          validation_data=(X_valid, y_valid))

Epoch 1/100
1852/1852 [==============================] - 4s 2ms/step - loss: 6.1039 - val_loss: 4.5182
Epoch 2/100
1852/1852 [==============================] - 3s 2ms/step - loss: 4.6334 - val_loss: 4.5242
Epoch 3/100
1852/1852 [==============================] - 3s 2ms/step - loss: 4.5459 - val_loss: 6.5747
Epoch 4/100
1852/1852 [==============================] - 3s 2ms/step - loss: 4.5020 - val_loss: 4.4181
Epoch 5/100
1852/1852 [==============================] - 3s 2ms/step - loss: 4.4698 - val_loss: 4.2956
Epoch 6/100
1852/1852 [==============================] - 3s 2ms/step - loss: 4.4318 - val_loss: 4.3882
Epoch 7/100
1852/1852 [==============================] - 3s 2ms/step - loss: 4.4144 - val_loss: 5.0692
Epoch 8/100
1852/1852 [==============================] - 3s 2ms/step - loss: 4.4047 - val_loss: 4.4415
Epoch 9/100
1852/1852 [==============================] - 3s 2ms/step - loss: 4.3920 - val_loss: 4.5989
Epoch 10/100
1852/1852 [==============================] - 3s 2ms/step - l

In [5]:
#Encode the sex with using OneHot Encoder
sex_data_test = encoder.fit_transform(test['Sex'].values.reshape(-1,1))
sex_df_test = pd.DataFrame(sex_data_test, columns=['Sex_F','Sex_I','Sex_M'])
sex_df_test = sex_df_test.set_index(test.index)

#Change the sex column on train data
test.drop('Sex',inplace=True,axis=1)
test = pd.concat([sex_df_test,test],axis=1)
test.head()

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Sex_F,Sex_I,Sex_M,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight
id,,,,,,,,,,
74051,0.0,1.0,0.0,1.0500,0.7625,0.2750,8.618248,3.657085,1.729319,2.721552
74052,0.0,1.0,0.0,1.1625,0.8875,0.2750,15.507176,7.030676,3.246018,3.968930
74053,1.0,0.0,0.0,1.2875,0.9875,0.3250,14.571643,5.556502,3.883882,4.819415
74054,1.0,0.0,0.0,1.5500,0.9875,0.3875,28.377849,13.380964,6.548735,7.030676
74055,0.0,1.0,0.0,1.1125,0.8500,0.2625,11.765042,5.528153,2.466407,3.331066


In [6]:
#predit sumbission result
age = model.predict(test)

1543/1543 [==============================] - 2s 1ms/step


In [7]:
#create a submission df and load to submit a result
submission = pd.DataFrame(age, index=test.index, columns=['Age'])

#Load submission to csv
submission.to_csv('/kaggle/working/submission.csv')